# Working with the NWB Format ("Neuroscience Without Borders")

Ref: https://pynwb.readthedocs.io/en/stable/index.html

In [1]:
#!conda install -c -y conda-forge pynwb
# %pip install pynwb

  Obtaining dependency information for pynwb from https://files.pythonhosted.org/packages/ce/3c/f2e265900fa86aadd5c111382149543c112e12c8caa4361de55db90dda37/pynwb-2.5.0-py3-none-any.whl.metadata
  Using cached pynwb-2.5.0-py3-none-any.whl.metadata (9.2 kB)
  Obtaining dependency information for hdmf>=3.9.0 from https://files.pythonhosted.org/packages/81/45/39e731f25936fb922b800bb5eb8871f8a59beea35b43de8cda951bf7099a/hdmf-3.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for ruamel-yaml>=0.16 from https://files.pythonhosted.org/packages/57/db/4b74a29f5fd175aea3ff0d95f8230d9bb8a54e38d963c6f96065b9e2eef3/ruamel.yaml-0.18.5-py3-none-any.whl.metadata
  Obtaining dependency information for ruamel.yaml.clib>=0.2.7 from https://files.pythonhosted.org/packages/ec/54/d8a795997921d87224c65d44499ca595a833093fb215b133f920c1062956/ruamel.yaml.clib-0.2.8-cp311-cp311-win_amd64.whl.metadata
Using cached pynwb-2.5.0-py3-none-any.whl (133 kB)
   ---------------------------------------- 

In [10]:
from datetime import datetime
from uuid import uuid4
from dateutil import tz

import numpy as np

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.file import Subject

## Creating the `NWBFile` object

In [4]:
nwbfile = NWBFile(
    
    # Required Inputs
    session_description="Mouse exploring an open field",
    identifier=str(uuid4()), 
    session_start_time=datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz("US/Pacific")), 

    # Optional Inputs
    session_id="session_1234", 
    experimenter=["Baggins, Bilbo"],
    lab="Bag End Laboratory",  
    institution="University of My Institution",  
    experiment_description="I went on an adventure to reclaim vast treasures.",  
    related_publications="DOI:10.1016/j.neuron.2016.12.011",  
)
nwbfile

root pynwb.file.NWBFile at 0x1987790799888
Fields:
  experiment_description: I went on an adventure to reclaim vast treasures.
  experimenter: ['Baggins, Bilbo']
  file_create_date: [datetime.datetime(2023, 12, 12, 14, 21, 33, 989254, tzinfo=tzlocal())]
  identifier: e83f547a-c693-487c-a4a2-8493d63bcdb2
  institution: University of My Institution
  lab: Bag End Laboratory
  related_publications: ['DOI:10.1016/j.neuron.2016.12.011']
  session_description: Mouse exploring an open field
  session_id: session_1234
  session_start_time: 2018-04-25 02:30:03-07:00
  timestamps_reference_time: 2018-04-25 02:30:03-07:00

## Add a `Subject` to a File

In [3]:
subject = Subject(
    subject_id="001",
    age="P90D",
    description="mouse 5",
    species="Mus musculus",
    sex="M",
)

nwbfile.subject = subject
subject

subject pynwb.file.Subject at 0x140679058853696
Fields:
  age: P90D
  age__reference: birth
  description: mouse 5
  sex: M
  species: Mus musculus
  subject_id: 001

## Add trials

For each trial, you can have a column, complete with metadata about the variable. Trial-level data can be made into a Pandas DataFrame with the `to_dataframe()` method.

In [8]:
nwbfile.add_trial_column(
    name="correct",
    description="whether the trial was correct",
)
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)
nwbfile.trials.to_dataframe()

,start_time,stop_time,correct
id,,,
0,1.0,5.0,True
1,6.0,10.0,False
2,1.0,5.0,True
3,6.0,10.0,False


## Save

In [11]:
with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    io.write(nwbfile)

## Load

In [12]:
with NWBHDF5IO("basics_tutorial.nwb", "r") as io:
    read_nwbfile = io.read()